# Classify critical configurations of logarithmic energy using the Hessian

12/24 - MVD.

# Packages

In [1]:
#############
# Packages
#############
import sympy as sp
# from itertools import permutations

# import numpy as np # numpy
import time
# import numpy.linalg as la # for the rank of a matrix

%run WfromX.py # load function that converts matrix X to matrix of points W


# Jacobian of auxiliary function

Define the auxiliary function $f: \mathbb{R}^3 \to \mathbb{R}$, such that
$$ f(w) := - \frac{2 w}{ \| w \|_2^2 } = -\frac{ 2 (x,y,z) }{ x^2 + y^2 + z^2 }, \quad w := (x,y,z)^T .$$

The Jacobian of $f$ is:
$$ J_f(w) := \frac{-2}{\| w \|_2^4} \begin{pmatrix}
		-x^2 + y^2 + z^2 & -2xy & -2xz \\
		 -2xy & x^2 - y^2 + z^2 & -2yz \\
		-2xz & -2yz & x^2 + y^2 - z^2 \\
\end{pmatrix} .$$
This may be rewritten as:
$$ J_f(w) := \frac{-2}{\| w \|_2^4} \begin{pmatrix}
		\| w \|_2^2 - 2 x^2 & -2xy & -2xz \\
		 -2xy & \| w \|_2^2 - 2 y^2 & -2yz \\
		-2xz & -2yz & \| w \|_2^2 - 2 z^2 \\
\end{pmatrix} = \frac{-2}{\| w \|_2^2} I_3 + \frac{4}{\| w \|_2^4} \begin{pmatrix}
		x^2 & xy & xz \\
		xy & y^2 & yz \\
		xz & yz & z^2 \\
\end{pmatrix} = \frac{-2}{\| w \|_2^2} I_3 + \frac{4}{\| w \|_2^4} w w^T .$$


In [2]:
#####################
# Jacobian of auxiliary function
# w is a row vector in R^d
#####################
def jacobianf(w):
    (null, d) = sp.shape(w) # number of rows (coordinates)

    # print('\nNumber of coordinates d =', d)

    Jf = ( -2 * sp.eye(d) / w.dot(w) ) + ( 4 * w.T * w / w.dot(w)**2 )

    return sp.simplify(Jf) # Jf


#################
# test in R^3
#################

#################
## Applied to w generic
#################
x,y,z = sp.var(['x','y','z'])

w = sp.Matrix( [ [x, y, z] ] ) # generic vector
sp.pprint(w)

Jf = jacobianf(w)
sp.pprint(Jf)

#################
## Applied to w-w0
#################
# x0,y0,z0 = sp.var(['x0','y0','z0'])

# w0 = sp.Matrix( [ [x0, y0, z0] ] ) # generic vector
# sp.pprint(w0)

# Jf0 = jacobianf(w-w0)
# sp.pprint(Jf0)



[x  y  z]
⎡  ⎛ 2    2    2⎞                                        ⎤
⎢2⋅⎝x  - y  - z ⎠        4⋅x⋅y               4⋅x⋅z       ⎥
⎢────────────────   ───────────────     ───────────────  ⎥
⎢              2                  2                   2  ⎥
⎢⎛ 2    2    2⎞     ⎛ 2    2    2⎞      ⎛ 2    2    2⎞   ⎥
⎢⎝x  + y  + z ⎠     ⎝x  + y  + z ⎠      ⎝x  + y  + z ⎠   ⎥
⎢                                                        ⎥
⎢                    ⎛   2    2    2⎞                    ⎥
⎢     4⋅x⋅y        2⋅⎝- x  + y  - z ⎠        4⋅y⋅z       ⎥
⎢───────────────   ──────────────────   ───────────────  ⎥
⎢              2                  2                   2  ⎥
⎢⎛ 2    2    2⎞     ⎛ 2    2    2⎞      ⎛ 2    2    2⎞   ⎥
⎢⎝x  + y  + z ⎠     ⎝x  + y  + z ⎠      ⎝x  + y  + z ⎠   ⎥
⎢                                                        ⎥
⎢                                        ⎛   2    2    2⎞⎥
⎢     4⋅x⋅z              4⋅y⋅z         2⋅⎝- x  - y  + z ⎠⎥
⎢───────────────    ───────────────    ───────

# Hessian of the Lagrangian

The gradient of the Lagrangian is:
$$ \frac{\partial L}{\partial w_i} = - \sum_{j=1, j \neq i}^n \frac{2 (w_i-w_j)}{ \|w_i - w_j \|_2^2 } + (n-1) w_i .$$

For the unit sphere $S^{d-1} \subset \mathbb{R}^d$, the Hessian of the Lagrangian is a matrix $H_L(w) \in \mathbb{R}^{d n \times d n}$. 

Using the jacobian of the auxiliary function $f$, matrix $H_L$ may be written as blocks of $d \times d$ matrices, given by:

1. Diagonal blocks:
$$ (H_L)_{ii} := \frac{\partial L}{\partial w_i^2} = \sum_{j=1, j \neq i} J_f(w_i-w_j) + (n-1) I_3, \quad \forall \ i .$$

2. Off diagonal blocks:
$$ (H_L)_{ij} := \frac{\partial L}{\partial w_i w_j} = -J_f(w_i-w_j), \quad \forall \ i \neq j .$$


In [3]:
#####################
# Hessian of the Lagrangian of the logarithmic energy problem on the sphere
#
# W is nxd, with coordinates of wi in row i
# HL is (n*d)x(n*d)
#####################
def hessL(W):
    (n, d) = sp.shape(W) # number of rows (points), and columns (coordinates)

    HL = sp.eye(n*d) # initial matrix
    
    for i in range(n): # each diagonal   
        wi = W[i,:]

        #######
        # diagonal block dxd
        #######
        HLaux = sp.zeros(d)
        for j in range(n) : # sum in each point != wi
            if j != i : # j!=i
                HLaux += jacobianf(wi - W[j,:])

        HL[d*i:d*(i+1), d*i:d*(i+1)] = HLaux + (n-1)*sp.eye(d)

        # print( ( HLaux + (n-1)*sp.eye(d) ).eigenvals() ) # eigenvals of the block

        #######
        # non diagonal blocks dxd
        #######
        HLaux = sp.zeros(d)
        for j in range(i+1,n) : # each point wj, j>i
            HLaux = -jacobianf(wi - W[j,:])
            
            HL[d*i:d*(i+1), d*j:d*(j+1)] = HLaux
            HL[d*j:d*(j+1), d*i:d*(i+1)] = HLaux # symmetric

        # sp.pprint(HLaux)
        
    return HL


## Test

In [4]:
##########
# Equator
##########

# # n=3 (global minima)
# W = sp.Matrix( [
#     [1, 0, 0],
#     [sp.cos(2*sp.pi/3), sp.sin(2*sp.pi/3), 0],
#     [sp.cos(4*sp.pi/3), sp.sin(4*sp.pi/3), 0],
# ] )
# sp.pprint(W)

# n=4
W = sp.Matrix( [
    [1, 0, 0],
    [0, 1, 0],
    [-1, 0, 0],
    [0, -1, 0]
] )

HL = hessL(W)

print('\nHessian HL of the Lagrangian:')
sp.pprint(HL)

print('\nIs HL symmetric?:', HL.T == HL ) # is symmetric

print('\nEigenvalues of HL:')
print( HL.eigenvals() )

print('\nDeterminant of HL:', HL.det() )



Hessian HL of the Lagrangian:
⎡7/2    0    0    0    1     0   -1/2   0    0    0    -1    0 ⎤
⎢                                                              ⎥
⎢ 0    5/2   0    1    0     0    0    1/2   0   -1    0     0 ⎥
⎢                                                              ⎥
⎢ 0     0   1/2   0    0     1    0     0   1/2   0    0     1 ⎥
⎢                                                              ⎥
⎢ 0     1    0   5/2   0     0    0    -1    0   1/2   0     0 ⎥
⎢                                                              ⎥
⎢ 1     0    0    0   7/2    0    -1    0    0    0   -1/2   0 ⎥
⎢                                                              ⎥
⎢ 0     0    1    0    0    1/2   0     0    1    0    0    1/2⎥
⎢                                                              ⎥
⎢-1/2   0    0    0    -1    0   7/2    0    0    0    1     0 ⎥
⎢                                                              ⎥
⎢ 0    1/2   0   -1    0     0    0    5/2   0    1    0   

# Projection of the Hessian onto the tangente space

We want to restrict the Hessian $H_L \in \mathbb{R}^{n d \times n d}$ to the tangent space of the product of spheres, as these are the admissible directions of movement.

For this we consider the "projected Hessian", which is a matrix $h_L \in \mathbb{R}^{n(d-1) \times n(d-1)}$, that acts on the coordinates of ambient vectors in a basis $B_t$ of the tangent space: $$ coord_{B_t}( P_T(H_L v) ) = h_L coord_{B_t} (v) ;$$ where $P_T(\cdot)$ denotes the orthogonal projection to the tangent space.

Using properties of linear algebra, it can be seen that: $$ h_L = Q_t^{T} H_L Q_t , \quad h_L \in \mathbb{R}^{n(d-1) \times n(d-1)} ;$$ where $Q_t := C [I]_{B_t} \in \mathbb{R}^{n d \times n(d-1)}$, $B_t$ is an **orthogonal** basis of the tangent space, and $C$ is the canonical basis of $\mathbb{R}^{nd}$.

## Basis of the tangent space

To construct an **orthogonal** basis of the tangent space of product of spheres, we join **orthogonal** basis of the tangent space to each sphere.

Given a point $w_i \in S^{d-1} \subset \mathbb{R}^d$, denote an orthogonal basis of the tangent space at this point as $$ \{ v_i^1, \ldots, v_i^{d-1} \} \subset \mathbb{R}^d .$$
These vectors may be embedded in $\mathbb{R}^{n d}$, assigning zero to each coordinate, except for the $d$ coordinates of the corresponding $v_i \in \mathbb{R}^d$, that go from the position $id$ to $(i+1)d-1$ (counting from index zero).

Note that the bases at two different points $w_i \neq w_j$ are orthogonal in $\mathbb{R}^{n d}$ by construction.

Thus, the union of these bases is an orthogonal base of the tangent space of the product of spheres: $$ B_t = \{ v_0^1, \ldots, v_0^{d-1}, \ldots , v_{n-1}^1, \ldots, v_{n-1}^{d-1} \} \subset \mathbb{R}^{n d} .$$


In [5]:
#########
# Project HL onto tangent space of product of spheres at W
########
def projHL(HL, W):    
    n, d = sp.shape(W) # number of points n and dimension d

    print(d)
        
    ###########
    # Orthonormal basis of tangent space at W
    ###########

    Bt = sp.zeros(n*d, n*(d-1)) # d-1 columns per point wi

    tGS = 0 # time GS
    tNS = 0 # time nullspace
    
    for i in range(n): # each point wi
        tini = time.time()
        Btaux = W[i,:].nullspace() # basis of tangent space at wi (d-1 vectors)
        tNS = tNS + ( time.time() - tini ) # adds nullspace time

        tini = time.time()
       
        Btaux = sp.GramSchmidt(Btaux, True) # orthonormalize vectors of basis
        Btaux = sp.Matrix( [ [ v for v in Btaux ] ] ) # matrix with basis as columns

        tGS = tGS + ( time.time() - tini ) # adds GS time

        Bt[d*i:d*(i+1), i*(d-1):(i+1)*(d-1)] = Btaux
    
    print('\nTime nullspace of all wi:', tNS )
    print('\nTime GS:', tGS )
    print('\nTime NS + GS:', tNS + tGS )

    # print('\nBasis of tangent space:')
    # sp.pprint(Bt)
    
    ###########
    # Project onto tangent space
    ###########
    tini = time.time()
    hL = Bt.T * HL * Bt # projected matrix (size n*(d-1) x n*(d-1))
    print('\nTime applying projection:', time.time() - tini )

    return hL, Bt # Bt is useful to convert eigenvectors of h_L to H_L
    


## Test

In [6]:
########
# Test
########

n = 4

# Equator (saddle point)
W = sp.Matrix( [
    [1, 0, 0],
    [0, 1, 0],
    [-1, 0, 0],
    [0, -1, 0],
] )
sp.pprint(W)

HL = hessL(W)
# print('\nHessian HL of the Lagrangian:')
# sp.pprint(HL)

print('\nProyecting HL onto tangent space...')
hL, Bt = projHL(HL, W)

print('\nOrthonormal basis of tangent space:')
print( sp.latex(Bt) )        
sp.pprint(Bt)        

print('\nHL proyected onto tangent space (via change of basis):')
sp.pprint(hL)

print('\nEigenvalues of projected Hessian (in numerical format):')
for l in hL.eigenvals() :
    print( sp.N(l) )

print('\nEigenvalues of projected Hessian (with multiplicity):')
print( hL.eigenvals() )

#############
# matrix HL
#############
print( HL.eigenvals() )



⎡1   0   0⎤
⎢         ⎥
⎢0   1   0⎥
⎢         ⎥
⎢-1  0   0⎥
⎢         ⎥
⎣0   -1  0⎦

Proyecting HL onto tangent space...
3

Time nullspace of all wi: 0.0008640289306640625

Time GS: 0.009511947631835938

Time NS + GS: 0.0103759765625

Time applying projection: 0.0002906322479248047

Orthonormal basis of tangent space:
\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix}\right]
⎡0  0  0  0  0  0  0  0⎤
⎢                      ⎥
⎢1  0  0  0  0  0  0  0⎥
⎢                      ⎥
⎢0  1  0  0  0  0  0  0⎥
⎢                      ⎥
⎢0  0  1  0  0  0  0  0⎥
⎢                      ⎥
⎢0  0  0  0  0  0  0  0⎥
⎢                      ⎥
⎢0  0  0  1  0  0  0

## Configuration Equator ($n$ equidistributed points at $z=0$)

In [124]:
#######
# Configuration Equator: n equidistributed points at z=0 in R^d; with d=2 (S^1) or d=3 (S^2).
#
# Input:
#  n: number of points, and
#  d: dimension of ambient space R^d.
#
# Output: matrix W with cartesian coordinates of w_k in row k
#######
def confEq(n, d=2):
    W = sp.eye(n, 2) # matrix with coordinates of wk in row k

    Theta = 2 * sp.pi / n # angle between contiguous points
    for k in range(n):
        W[k,:] = sp.Matrix( [ [ sp.cos(k * Theta), sp.sin(k * Theta) ] ] )

    if d==3: # S^2
        W = W.hstack(W, sp.zeros(n,1) ) # adds 0 coordinate at the end of each point

    return W



## Configuration 1:(n-2):1

In [125]:
#######
# Configuration 1:(n-2):1 in R^d, with d=3 (S^2) or d=4 (S^3).
#
# Input:
#  n: number of points, and
#  d: dimension of ambient space R^d.
#
# Output: matrix W with cartesian coordinates of w_k in row k
#######
def conf1N1(n, d=3):
    W = sp.eye(n, 3) # matrix with coordinates of wk in row k

    W[0,:] = sp.Matrix( [ [0, 0, 1] ] ) # north pole
    W[n-1,:] = sp.Matrix( [ [0, 0, -1] ] ) # south pole
    
    #########
    # n-2 equidistributed points at z=0
    #########
    Theta = 2 * sp.pi / (n-2) # angle between contiguous points
    
    for k in range(n-2):
        W[k+1,:d] = sp.Matrix( [ [ sp.cos(k * Theta), sp.sin(k * Theta), 0] ] )
    
    if d==4: # S^3
        W = W.hstack(W, sp.zeros(n,1)) # adds 0 coordinate at the end of each point

    return W


## Configuration 1:(n-1)

In [126]:
#######
# Configuration 1:(n-1) in R^d, with d=3 (S^2) or d=4 (S^3).
#
# Input:
#  n: number of points, and
#  d: dimension of ambient space R^d.
#
# Output: matrix W with cartesian coordinates of w_k in row k
#######
def conf1N(n, d=3):
    z0 = -sp.Rational(1, n-1) # height of the plane with n-1 points equidistirbuted
    R = sp.sqrt( 1 - z0**2 ) # radius of cricle at height z
    
    W = sp.eye(n,3) # matrix with coordinates of wk in row k
    W[0,:] = sp.Matrix( [ [0, 0, 1] ] ) # north pole
    
    #########
    # n-1 equidistributed points at z=z0
    #########
    Theta = 2 * sp.pi / (n-1) # angle between contiguous points
    
    for k in range(n-1):
        W[k+1,:] = sp.Matrix( [ [ R * sp.cos(k * Theta), R * sp.sin(k * Theta), z0] ] )

    if d==4: # S^3
        W = W.hstack(W, sp.zeros(n,1)) # adds 0 coordinate at the end of each point
    
    return W


## Configuration $(n-1)$-simplex (global minima in $\mathbb{R}^{n-1}$)

In [8]:
#######
# Configuration (n-1)-simplex in R^d, with n-1<=d.
#
# For any pair of points wi and wj, we have: wi^T wj = -1/(n-1).
#
# Input:
#  n: number of points, and
#  d: dimension of ambient space R^d, with n-1<=d.
#
# Output: matrix W with cartesian coordinates of w_k in row k
#######
def confSimplex(n, d=3):
    t = -sp.Rational(1, n-1) # dot product of any pair of points
    
    # nxn matrix with 1s in its diagonal and t outside its diagonal
    X = t * sp.ones(n) - (t-1) * sp.eye(n)
    
    W = WfromX(X, d) # matrix of points from matrix of dot products

    return W


# $n=4$ points

## Equator

In [12]:
##########
# Equator
##########
n = 4 # number of points
d = 3 # points in R^d

W = confEq(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)

HL = hessL(W)
print('\nHessian of Lagrangian:')
sp.pprint(HL)


print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )



Configuration of points:
⎡1   0   0⎤
⎢         ⎥
⎢0   1   0⎥
⎢         ⎥
⎢-1  0   0⎥
⎢         ⎥
⎣0   -1  0⎦

Hessian of Lagrangian:
⎡7/2    0    0    0    1     0   -1/2   0    0    0    -1    0 ⎤
⎢                                                              ⎥
⎢ 0    5/2   0    1    0     0    0    1/2   0   -1    0     0 ⎥
⎢                                                              ⎥
⎢ 0     0   1/2   0    0     1    0     0   1/2   0    0     1 ⎥
⎢                                                              ⎥
⎢ 0     1    0   5/2   0     0    0    -1    0   1/2   0     0 ⎥
⎢                                                              ⎥
⎢ 1     0    0    0   7/2    0    -1    0    0    0   -1/2   0 ⎥
⎢                                                              ⎥
⎢ 0     0    1    0    0    1/2   0     0    1    0    0    1/2⎥
⎢                                                              ⎥
⎢-1/2   0    0    0    -1    0   7/2    0    0    0    1     0 ⎥
⎢                    

## Tetrahedron (global minima)

In [13]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 4 # number of points
d = 3 # points in R^d

W = conf1N(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)

HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)
    
print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )




Configuration of points:
⎡ 0     0     1  ⎤
⎢                ⎥
⎢2⋅√2            ⎥
⎢────   0    -1/3⎥
⎢ 3              ⎥
⎢                ⎥
⎢-√2    √6       ⎥
⎢────   ──   -1/3⎥
⎢ 3     3        ⎥
⎢                ⎥
⎢-√2   -√6       ⎥
⎢────  ────  -1/3⎥
⎣ 3     3        ⎦

HL proyected onto tangent space:
3

Time nullspace of all wi: 0.0013422966003417969

Time GS: 0.007017612457275391

Time NS + GS: 0.008359909057617188

Time applying projection: 0.0042345523834228516
⎡                          3⋅√3        -3⋅√3         ⎤
⎢ 3/2      0      0   3/4  ────  -3/8  ──────   -3/8 ⎥
⎢                           8            8           ⎥
⎢                                                    ⎥
⎢                                3⋅√3          -3⋅√3 ⎥
⎢  0      3/2    3/4   0   3/8   ────   3/8    ──────⎥
⎢                                 8              8   ⎥
⎢                                                    ⎥
⎢  0      3/4    3/2   0   3/4    0     3/4      0   ⎥
⎢                              

# $n=5$ points

## Equator

Calculating eigenvalues takes some time (less than 1 minute).

It is necessary to apply sp.simplify() to $h_L$. Otherwise calculating eigenvalues takes too much time.

In [24]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 5 # number of points
d = 3 # points in R^d

print('\nPoints in R^d, with d =', d)

W = confEq(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)

#######
# Eigenvalues of projected Hessian of Lagrangian
#######

print('\nHessian of Lagrangian:')
start = time.time()
HL = hessL(W)
# sp.pprint(HL)
print('\nTime calculating Hessian:', time.time() - start)

print('\nHessian proyected onto tangent space:')
start = time.time()

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL) # without sp.simplify, calculating eigenvalues takes too much time

print('\nTime projecting Hessian:', time.time() - start)
# sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
start = time.time()
print( hL.eigenvals() )
print('\nTime eigenvalues:', time.time() - start)


#############
# matrix HL (muy costoso en sympy).
#############
# print('\nEigenvalues of Hessian with multiplicity:')
# start = time.time()
# print( HL.eigenvals() )
# print('\nTime eigenvalues:', time.time() - start)


# t = -2 # eigenvalue to test
# print( ( hL - t * sp.eye( n*(d-1), n*(d-1) ) ).det() )

#######
# Search negative direction moving wi and wj
#######
# i = 0; j = 1 # pair of points to move
# n, d = sp.shape(W) # number of points n and dimension d
# hLij = projHLij(HL, i, j, d) # HLij projected to tangent space

# hLij = sp.cancel(hLij)
# sp.pprint(hLij)

# print('\nEigenvalues of projected matrix (in numeric format):')
# for l in hLij.eigenvals():
#     print( sp.N(l) )



Points in R^d, with d = 3

Configuration of points:
⎡   1            0        0⎤
⎢                          ⎥
⎢              ________    ⎥
⎢  1   √5     ╱ √5   5     ⎥
⎢- ─ + ──    ╱  ── + ─    0⎥
⎢  4   4   ╲╱   8    8     ⎥
⎢                          ⎥
⎢              ________    ⎥
⎢  √5   1     ╱ 5   √5     ⎥
⎢- ── - ─    ╱  ─ - ──    0⎥
⎢  4    4  ╲╱   8   8      ⎥
⎢                          ⎥
⎢               ________   ⎥
⎢  √5   1      ╱ 5   √5    ⎥
⎢- ── - ─  -  ╱  ─ - ──   0⎥
⎢  4    4   ╲╱   8   8     ⎥
⎢                          ⎥
⎢               ________   ⎥
⎢  1   √5      ╱ √5   5    ⎥
⎢- ─ + ──  -  ╱  ── + ─   0⎥
⎣  4   4    ╲╱   8    8    ⎦

Hessian of Lagrangian:

Time calculating Hessian: 2.8815908432006836

Hessian proyected onto tangent space:
3

Time nullspace of all wi: 0.00853729248046875

Time GS: 0.012966394424438477

Time NS + GS: 0.021503686904907227

Time applying projection: 0.3259565830230713

Time projecting Hessian: 3.336862087249756

Eigenvalues of project

## Configuration 1:3:1 (global minima)

In [25]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 5 # number of points
d = 3 # points in R^d

W = conf1N1(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)

HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL)

sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
# print( HL.eigenvals() )



Configuration of points:
⎡ 0     0    1 ⎤
⎢              ⎥
⎢ 1     0    0 ⎥
⎢              ⎥
⎢       √3     ⎥
⎢-1/2   ──   0 ⎥
⎢       2      ⎥
⎢              ⎥
⎢      -√3     ⎥
⎢-1/2  ────  0 ⎥
⎢       2      ⎥
⎢              ⎥
⎣ 0     0    -1⎦

HL proyected onto tangent space:
3

Time nullspace of all wi: 0.0007834434509277344

Time GS: 0.004856586456298828

Time NS + GS: 0.0056400299072265625

Time applying projection: 0.00203704833984375
⎡                       √3         -√3                   ⎤
⎢ 2     0     0    1    ──   -1/2  ────  -1/2  1/2    0  ⎥
⎢                       2           2                    ⎥
⎢                                                        ⎥
⎢                             √3         -√3             ⎥
⎢ 0     2     1    0   1/2    ──   1/2   ────   0    1/2 ⎥
⎢                             2           2              ⎥
⎢                                                        ⎥
⎢ 0     1    4/3   0   2/3    0    2/3    0     0     1  ⎥
⎢                     

## Configuration 1:4

In [26]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 5 # number of points
d = 3 # points in R^d

W = conf1N(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)

HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL)

sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )




Configuration of points:
⎡  0      0     1  ⎤
⎢                  ⎥
⎢ √15              ⎥
⎢ ───     0    -1/4⎥
⎢  4               ⎥
⎢                  ⎥
⎢        √15       ⎥
⎢  0     ───   -1/4⎥
⎢         4        ⎥
⎢                  ⎥
⎢-√15              ⎥
⎢─────    0    -1/4⎥
⎢  4               ⎥
⎢                  ⎥
⎢       -√15       ⎥
⎢  0    ─────  -1/4⎥
⎣         4        ⎦

HL proyected onto tangent space:
3

Time nullspace of all wi: 0.0015954971313476562

Time GS: 0.006155252456665039

Time NS + GS: 0.007750749588012695

Time applying projection: 0.0011875629425048828
⎡ 2     0     0    4/5   4/5    0     0    -4/5  4/5    0  ⎤
⎢                                                          ⎥
⎢ 0     2    4/5    0     0    4/5   4/5    0     0    -4/5⎥
⎢                                                          ⎥
⎢                         16                     -16       ⎥
⎢ 0    4/5   8/3    0     ──    0    8/15   0    ────   0  ⎥
⎢                         15                      

## 4-simplex in $S^3 \subset \mathbb{R}^4$ (global minima)

In [27]:
# t = -sp.Rational(1,4)

# X = sp.Matrix([
#     [1, t, t, t, t],
#     [t, 1, t, t, t],
#     [t, t, 1, t, t],
#     [t, t, t, 1, t],
#     [t, t, t, t, 1]
# ])

# d = X.rank() # d=4
# W = WfromX(X, d) # matrix of points from matrix of dot products

##########
# Cartesian coordinates of points on the sphere
##########
n = 5 # number of points
d = 4 # points in R^d

W = confSimplex(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)


# print('\nHessian of Lagrangian:')
HL = hessL(W)
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL)

sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )




Eigenvalues of X
⎡0   0    0    0    0 ⎤
⎢                     ⎥
⎢0  5/4   0    0    0 ⎥
⎢                     ⎥
⎢0   0   5/4   0    0 ⎥
⎢                     ⎥
⎢0   0    0   5/4   0 ⎥
⎢                     ⎥
⎣0   0    0    0   5/4⎦

Eigenvectors of X
⎡1  -1  -1  -1  -1⎤
⎢                 ⎥
⎢1  1   0   0   0 ⎥
⎢                 ⎥
⎢1  0   1   0   0 ⎥
⎢                 ⎥
⎢1  0   0   1   0 ⎥
⎢                 ⎥
⎣1  0   0   0   1 ⎦

Non-null eigenvalues D2:
⎡5/4   0    0    0 ⎤
⎢                  ⎥
⎢ 0   5/4   0    0 ⎥
⎢                  ⎥
⎢ 0    0   5/4   0 ⎥
⎢                  ⎥
⎣ 0    0    0   5/4⎦

Corresponding orthogonal matrix of eigenvectors Q:
⎡-√2   -√6   -√3   -√5 ⎤
⎢────  ────  ────  ────⎥
⎢ 2     6     6     10 ⎥
⎢                      ⎥
⎢ √2   -√6   -√3   -√5 ⎥
⎢ ──   ────  ────  ────⎥
⎢ 2     6     6     10 ⎥
⎢                      ⎥
⎢       √6   -√3   -√5 ⎥
⎢ 0     ──   ────  ────⎥
⎢       3     6     10 ⎥
⎢                      ⎥
⎢             √3   -√5 ⎥
⎢ 0     0     ── 

# $n=6$ points

## Equator

In [28]:
##########
# Equator
##########
n = 6 # number of points
d = 3 # points in R^d

W = confEq(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)


# print('\nHessian of Lagrangian:')
HL = hessL(W)
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL)

sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )



Configuration of points:
⎡ 1     0    0⎤
⎢             ⎥
⎢       √3    ⎥
⎢1/2    ──   0⎥
⎢       2     ⎥
⎢             ⎥
⎢       √3    ⎥
⎢-1/2   ──   0⎥
⎢       2     ⎥
⎢             ⎥
⎢ -1    0    0⎥
⎢             ⎥
⎢      -√3    ⎥
⎢-1/2  ────  0⎥
⎢       2     ⎥
⎢             ⎥
⎢      -√3    ⎥
⎢1/2   ────  0⎥
⎣       2     ⎦

HL proyected onto tangent space:
3

Time nullspace of all wi: 0.0010221004486083984

Time GS: 0.006196737289428711

Time NS + GS: 0.007218837738037109

Time applying projection: 0.0021178722381591797
⎡35/6   0     -2    0    2/3    0    1/2    0    2/3    0     -2    0  ⎤
⎢                                                                      ⎥
⎢ 0    -5/6   0     2     0    2/3    0    1/2    0    2/3    0     2  ⎥
⎢                                                                      ⎥
⎢ -2    0    35/6   0     2     0    2/3    0    1/2    0    -2/3   0  ⎥
⎢                                                                      ⎥
⎢ 0     2     0    -5/6   0    

## Configuration 1:4:1 (global minima in $S^2$, saddle in $S^d$, $d>2$)

In [29]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 6 # number of points
d = 3 # points in R^d

W = conf1N1(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)


# print('\nHessian of Lagrangian:')
HL = hessL(W)
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')

hL, Bt = projHL(HL, W)
hL = sp.simplify(hL)

sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )



Configuration of points:
⎡0   0   1 ⎤
⎢          ⎥
⎢1   0   0 ⎥
⎢          ⎥
⎢0   1   0 ⎥
⎢          ⎥
⎢-1  0   0 ⎥
⎢          ⎥
⎢0   -1  0 ⎥
⎢          ⎥
⎣0   0   -1⎦

HL proyected onto tangent space:
3

Time nullspace of all wi: 0.0005810260772705078

Time GS: 0.005440235137939453

Time NS + GS: 0.006021261215209961

Time applying projection: 0.00019240379333496094
⎡5/2   0    0    1    1    0    0   -1    1    0   1/2   0 ⎤
⎢                                                          ⎥
⎢ 0   5/2   1    0    0    1    1    0    0   -1    0   1/2⎥
⎢                                                          ⎥
⎢ 0    1   5/2   0    1    0   1/2   0   -1    0    0    1 ⎥
⎢                                                          ⎥
⎢ 1    0    0   5/2   0    1    0   1/2   0    1   -1    0 ⎥
⎢                                                          ⎥
⎢ 1    0    1    0   5/2   0   -1    0   1/2   0    1    0 ⎥
⎢                                                          ⎥
⎢ 0    1    0    1 

## Configuration 1:5

In [128]:
##########
# Cartesian coordinates of points on the sphere
##########
n = 6 # number of points
d = 3 # points in R^d

W = conf1N(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)



Configuration of points:
⎡       0                  0            1  ⎤
⎢                                          ⎥
⎢     2⋅√6                                 ⎥
⎢     ────                 0           -1/5⎥
⎢      5                                   ⎥
⎢                                          ⎥
⎢                           ________       ⎥
⎢     ⎛  1   √5⎞           ╱ √5   5        ⎥
⎢2⋅√6⋅⎜- ─ + ──⎟   2⋅√6⋅  ╱  ── + ─        ⎥
⎢     ⎝  4   4 ⎠        ╲╱   8    8        ⎥
⎢───────────────   ─────────────────   -1/5⎥
⎢       5                  5               ⎥
⎢                                          ⎥
⎢                           ________       ⎥
⎢     ⎛  √5   1⎞           ╱ 5   √5        ⎥
⎢2⋅√6⋅⎜- ── - ─⎟   2⋅√6⋅  ╱  ─ - ──        ⎥
⎢     ⎝  4    4⎠        ╲╱   8   8         ⎥
⎢───────────────   ─────────────────   -1/5⎥
⎢       5                  5               ⎥
⎢                                          ⎥
⎢                           ________       ⎥
⎢     ⎛  √5   1⎞           ╱ 

### Projected Hessian

In [129]:

print('\nHessian of Lagrangian:')
start = time.time()
HL = hessL(W)
# sp.pprint(HL)
print('\nTime calculating Hessian:', time.time() - start)

print('\nProyecting Hessian onto tangent space...')
start = time.time()
hL, Bt = projHL(HL, W)
print('\nTime projecting Hessian:', time.time() - start)

# print('\nSimplifying projected hessian...')
# start = time.time()
# hL = sp.simplify(hL) # takes too much time (waited 10 minutes in minipc)
# print('\nTime simplifying:', time.time() - start)

# sp.pprint(hL)



Hessian of Lagrangian:

Time calculating Hessian: 5.986756324768066

Proyecting Hessian onto tangent space...
3

Time nullspace of all wi: 0.012016534805297852

Time GS: 0.05964541435241699

Time NS + GS: 0.07166194915771484

Time applying projection: 0.16767168045043945

Time projecting Hessian: 0.23976778984069824


### Eigenvalues of projected Hessian (numerical)

In [16]:
import numpy as np
from numpy import linalg as la

hLN = np.array(hL).astype(np.float64) # convert hL to numpy numerical matrix

print('\nEigenvalues...')

start = time.time()
print( la.eigvals(hLN) ) # one of them is: -1.25 = -5/4 (double)
print('\nTime eigenvalues:', time.time() - start)



Eigenvalues...
[ 0.00000000e+00  2.50000000e+00  5.00000000e+00  6.25000000e+00
  5.00000000e+00  2.50000000e+00 -1.25000000e+00 -1.25000000e+00
 -7.63626302e-16  7.03874646e-16  6.25000000e+00  5.00000000e+00]

Time eigenvalues: 0.009139537811279297


### Find positive and negative tangent direction

In [143]:
dirNegative = True # True for negative tangent direction (False for positive)

n, d = sp.shape(W) # number of points and dimension of ambient space R^d

#######
# Tangent vector that moves w2 upwards
#######
w2 = W[1,:] # point w2
# sp.pprint(w2)
# print(sp.latex(w2))

# projection of (0,0,-1) onto tangent space of w2 (orthogonal to w2)
vT2 = sp.Matrix( [ [0,0,1] ] ) - w2[2] * w2
vT2 = vT2.T # column vector

# sp.pprint(vT2)

#######
# Tangent vector that moves w3 downwards (for a negative direction) or upwards (for a positive direction)
#######
w3 = W[2,:] # point w3
# sp.pprint(w3)
# print(sp.latex(w3))

if dirNegative == True :
    # projection of (0,0,-1) onto tangent space of w3 (orthogonal to w3)
    vT3 = sp.Matrix( [ [0,0,-1] ] ) - w3[2] * w3
else:
    # projection of (0,0,1) onto tangent space of w3 (orthogonal to w3)
    vT3 = sp.Matrix( [ [0,0,1] ] ) - w3[2] * w3

vT3 = vT3.T # column vector

# sp.pprint(vT3)

vT = sp.Matrix( [vT2, vT3] ) # union of tangent vectors

print('\nTangent vector of movement:')
sp.pprint(vT.T) # print as row vector
print( '\nLatex:', sp.latex(vT) )

######
# Evaluates Hessian HL in tangent vector (non projected Hessian as our vectors are in ambient space)
######
vaux = HL[:, d:3*d] * vT # multiply right
vHLv = vT.T * sp.Matrix( vaux[d:3*d] ) # multiply left

vHLv = sp.simplify(vHLv) # simplify expression

print('\nHessian evaluated in tangent vector:')

sp.pprint(vHLv) # result symbolic
print( '\nNumeric:', sp.N(vHLv)[0] ) # result numeric
print( '\nLatex:', sp.latex(vHLv) ) # result latex




Tangent vector of movement:
⎡                                       ________      ⎤
⎢                  ⎛  1   √5⎞          ╱ √5   5       ⎥
⎢             2⋅√6⋅⎜- ─ + ──⎟  2⋅√6⋅  ╱  ── + ─       ⎥
⎢2⋅√6     24       ⎝  4   4 ⎠       ╲╱   8    8   -26 ⎥
⎢────  0  ──  ───────────────  ─────────────────  ────⎥
⎣ 25      25        25                25           25 ⎦

Latex: \left[\begin{matrix}\frac{2 \sqrt{6}}{25}\\0\\\frac{24}{25}\\\frac{2 \sqrt{6} \left(- \frac{1}{4} + \frac{\sqrt{5}}{4}\right)}{25}\\\frac{2 \sqrt{6} \sqrt{\frac{\sqrt{5}}{8} + \frac{5}{8}}}{25}\\- \frac{26}{25}\end{matrix}\right]

Hessian evaluated in tangent vector:
⎡                ____________      ⎤
⎢  263⋅√5   13⋅╲╱ 10⋅√5 + 30    37 ⎥
⎢- ────── + ───────────────── + ───⎥
⎣   625            625          125⎦

Numeric: -0.494427190999916

Latex: \left[\begin{matrix}- \frac{263 \sqrt{5}}{625} + \frac{13 \sqrt{10 \sqrt{5} + 30}}{625} + \frac{37}{125}\end{matrix}\right]


## Configuration 3:3 with null phase shift

In [144]:
##########
# Configuration 3:3 with null phase (60 permutations)
##########
z0 = sp.sqrt( ( -3 + 2 * sp.sqrt(6) ) / 5 ) # height
R = sp.sqrt(1 - z0**2) # radius

w1 = [ R, 0, z0 ]
w2 = [ -R/2, ( R * sp.sqrt(3) ) / 2, z0 ]
w3 = [ -R/2, -( R * sp.sqrt(3) ) / 2, z0 ]

w4 = [ R, 0, -z0 ]
w5 = [ -R/2, ( R * sp.sqrt(3) ) / 2, -z0 ]
w6 = [ -R/2, -( R * sp.sqrt(3) ) / 2, -z0 ]

W = sp.Matrix( [ w1, w2, w3, w4, w5, w6 ] )

print('\nConfiguration of points:')
sp.pprint(W)




Configuration of points:
⎡     __________                            ____________ ⎤
⎢    ╱ 8   2⋅√6                            ╱   3   2⋅√6  ⎥
⎢   ╱  ─ - ────             0             ╱  - ─ + ────  ⎥
⎢ ╲╱   5    5                           ╲╱     5    5    ⎥
⎢                                                        ⎥
⎢     __________           __________                    ⎥
⎢    ╱ 8   2⋅√6           ╱ 8   2⋅√6                     ⎥
⎢-  ╱  ─ - ────     √3⋅  ╱  ─ - ────        ____________ ⎥
⎢ ╲╱   5    5          ╲╱   5    5         ╱   3   2⋅√6  ⎥
⎢────────────────   ─────────────────     ╱  - ─ + ────  ⎥
⎢       2                   2           ╲╱     5    5    ⎥
⎢                                                        ⎥
⎢     __________           __________                    ⎥
⎢    ╱ 8   2⋅√6           ╱ 8   2⋅√6                     ⎥
⎢-  ╱  ─ - ────    -√3⋅  ╱  ─ - ────        ____________ ⎥
⎢ ╲╱   5    5          ╲╱   5    5         ╱   3   2⋅√6  ⎥
⎢────────────────  ───────────

### Hessian and projected hessian

In [145]:
print('\nCalculating Hessian of Lagrangian...')
start = time.time()
HL = hessL(W)
print('\nTime calculating Hessian:', time.time() - start)
# sp.pprint(HL)

print('\nProyecting Hessian onto tangent space...')
start = time.time()
hL, Bt = projHL(HL, W)
print('\nTime projecting Hessian:', time.time() - start)

print('\nSimplifying projected Hessian...') # takes some time (40s in minipc)
start = time.time()
hL = sp.simplify(hL)
print('\nTime simplifying projected Hessian:', time.time() - start)

# sp.pprint(hL)



Calculating Hessian of Lagrangian...

Time calculating Hessian: 2.7739977836608887

Proyecting Hessian onto tangent space...
3

Time nullspace of all wi: 0.011271476745605469

Time GS: 0.012051582336425781

Time NS + GS: 0.02332305908203125

Time applying projection: 0.04581260681152344

Time projecting Hessian: 0.06966948509216309

Simplifying projected Hessian...

Time simplifying projected Hessian: 37.259151220321655


### Eigenvalues of projected Hessian (numerical)

In [99]:
import numpy as np
from numpy import linalg as la

hLN = np.array(hL).astype(np.float64) # converts to numerical matrix using numpy

print('\nEigenvalues...')

start = time.time()
print( la.eigvals(hLN) ) # hay uno que es -1.25 = -5/4 (doble)
print('\nTime eigenvalues:', time.time() - start)

# Exact eigenvalues: 5 triple, 0 triple.



Eigenvalues...
[ 5.00000000e+00+0.00000000e+00j  3.55051026e+00+0.00000000e+00j
 -7.97958971e-01+0.00000000e+00j  1.44948974e+00+0.00000000e+00j
  3.20697032e-17+1.15473044e-16j  3.20697032e-17-1.15473044e-16j
  1.44948974e+00+0.00000000e+00j  3.55051026e+00+0.00000000e+00j
  5.79795897e+00+0.00000000e+00j  5.00000000e+00+0.00000000e+00j
  5.00000000e+00+0.00000000e+00j -2.22044605e-16+0.00000000e+00j]

Time eigenvalues: 0.000370025634765625


### Find tangent direction of negative eigenvalue

In [148]:
n, d = sp.shape(W) # number of points and dimension of ambient space R^d

########
# Tangent vector of movement.
# Obtained from counter-clockwise rotation with respect to the z-axis of the upper triangle (first three points).
# The lower triangle remains fixed (last three points of the configuration).
########
vT = sp.Matrix( [ [] ] ) # null matrix
for i in range(3): # each of the first three points
    wAux = W[i,:] # coordinates of point wi
    print( '\nLatex:', sp.latex(wAux) )

    # tangent vector in R^3 associated to the counter-clockwise rotation with respect to the z-axis 
    vAux = sp.Matrix( [ -wAux[1], wAux[0], 0 ] ).T # (x,y,z) --> (-y,x,0)
    # sp.pprint(vAux)

    vT = vT.hstack(vT, vAux) # appends to tangent vector in R^{3d}

vT = vT.T # column vector in R^{3d}

print('\nTangent vector of movement:')
sp.pprint(vT.T) # print as row vector
print( '\nLatex:', sp.latex(vT) )


######
# Evaluates Hessian HL in tangent vector (non projected Hessian as our vectors are in ambient space)
######
vaux = HL[:, 0:3*d] * vT # multiply right (only first 3d columns involved)
vHLv = vT.T * sp.Matrix( vaux[0:3*d] ) # multiply left (only first 3d coordinates involved)

vHLv = sp.simplify(vHLv) # simplify expression

print('\nHessian evaluated in tangent vector:')
sp.pprint(vHLv) # result symbolic
sp.N(vHLv) # result numeric




Latex: \left[\begin{matrix}\sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}} & 0 & \sqrt{- \frac{3}{5} + \frac{2 \sqrt{6}}{5}}\end{matrix}\right]

Latex: \left[\begin{matrix}- \frac{\sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}}}{2} & \frac{\sqrt{3} \sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}}}{2} & \sqrt{- \frac{3}{5} + \frac{2 \sqrt{6}}{5}}\end{matrix}\right]

Latex: \left[\begin{matrix}- \frac{\sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}}}{2} & - \frac{\sqrt{3} \sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}}}{2} & \sqrt{- \frac{3}{5} + \frac{2 \sqrt{6}}{5}}\end{matrix}\right]

Tangent vector of movement:
⎡                              __________        __________             ______ ↪
⎢                             ╱ 8   2⋅√6        ╱ 8   2⋅√6             ╱ 8   2 ↪
⎢       __________     -√3⋅  ╱  ─ - ────    -  ╱  ─ - ────       √3⋅  ╱  ─ - ─ ↪
⎢      ╱ 8   2⋅√6          ╲╱   5    5       ╲╱   5    5            ╲╱   5     ↪
⎢0    ╱  ─ - ────   0  ───────────────────  ────────────────  0  ───────────── ↪
⎣   ╲

Matrix([[-0.742346141747671]])

### Find tangent direction of positive eigenvalue

In [154]:
n, d = sp.shape(W) # number of points and dimension of ambient space R^d

########
# Tangent vector of movement.
########
w0 = W[0,:] # coordinates of point w0
vT = sp.Matrix([[0,0,1]]) - w0[2] * w0 # rojection of (0,0,1) onto orthogonal of w0 (tangent space)

vT = vT.T # column vector in R^{d}

print('\nTangent vector of movement:')
sp.pprint(vT.T) # print as row vector
print( '\nLatex:', sp.latex(vT) )


######
# Evaluates Hessian HL in tangent vector (non projected Hessian as our vectors are in ambient space)
######
vaux = HL[:, 0:d] * vT # multiply right (only first d columns involved)
vHLv = vT.T * sp.Matrix( vaux[0:d] ) # multiply left (only first d coordinates involved)

vHLv = sp.simplify(vHLv) # simplify expression

print('\nHessian evaluated in tangent vector:')
sp.pprint(vHLv) # result symbolic
print( '\nNumeric:', sp.N(vHLv)[0] ) # result numeric
print( '\nLatex:', sp.latex(vHLv) )



Tangent vector of movement:
⎡     ____________     __________             ⎤
⎢    ╱   3   2⋅√6     ╱ 8   2⋅√6      8   2⋅√6⎥
⎢-  ╱  - ─ + ──── ⋅  ╱  ─ - ────   0  ─ - ────⎥
⎣ ╲╱     5    5    ╲╱   5    5        5    5  ⎦

Latex: \left[\begin{matrix}- \sqrt{- \frac{3}{5} + \frac{2 \sqrt{6}}{5}} \sqrt{\frac{8}{5} - \frac{2 \sqrt{6}}{5}}\\0\\\frac{8}{5} - \frac{2 \sqrt{6}}{5}\end{matrix}\right]

Hessian evaluated in tangent vector:
⎡                 _____________        _____________         ⎤
⎢  1016   32⋅√6⋅╲╱ 217 - 88⋅√6    72⋅╲╱ 217 - 88⋅√6    143⋅√6⎥
⎢- ──── - ───────────────────── + ────────────────── + ──────⎥
⎣   15             25                     25             5   ⎦

Numeric: 2.01513589501620

Latex: \left[\begin{matrix}- \frac{1016}{15} - \frac{32 \sqrt{6} \sqrt{217 - 88 \sqrt{6}}}{25} + \frac{72 \sqrt{217 - 88 \sqrt{6}}}{25} + \frac{143 \sqrt{6}}{5}\end{matrix}\right]


## Solutions with $rank(X) \geq 4$

## Configuration with $x_{45} = -1/3$ (New 1)

In [45]:
A = -sp.Rational(1, 3) # -1/3
B = 0
C = -1

X = sp.Matrix( [
    [1, A, B, B, A, A ],
    [0, 1, B, B, A, A ],
    [0, 0, 1, C, B, B ],
    [0, 0, 0, 1, B, B ],
    [0, 0, 0, 0, 1, A ],
    [0, 0, 0, 0, 0, 1 ]
] )

X = (X + X.T) - sp.eye(6) # simetriza la matriz

#######
# Matrix of coordinates on the sphere
#######
d = X.rank() # d=4

W = WfromX(X, d) # S^3 ?is local minima?
# W = WfromX(X, d+1) # S^4 is saddle

print('\nConfiguration of points:')
sp.pprint(W)

########
# Search for negative feasible direction
#######

HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicities:')
print( hL.eigenvals() )

#############
# matrix HL
#############
print( HL.eigenvals() )



Eigenvalues of X
⎡0  0   0    0    0   0⎤
⎢                      ⎥
⎢0  0   0    0    0   0⎥
⎢                      ⎥
⎢0  0  4/3   0    0   0⎥
⎢                      ⎥
⎢0  0   0   4/3   0   0⎥
⎢                      ⎥
⎢0  0   0    0   4/3  0⎥
⎢                      ⎥
⎣0  0   0    0    0   2⎦

Eigenvectors of X
⎡0  1  -1  -1  -1  0 ⎤
⎢                    ⎥
⎢0  1  1   0   0   0 ⎥
⎢                    ⎥
⎢1  0  0   0   0   -1⎥
⎢                    ⎥
⎢1  0  0   0   0   1 ⎥
⎢                    ⎥
⎢0  1  0   1   0   0 ⎥
⎢                    ⎥
⎣0  1  0   0   1   0 ⎦

Non-null eigenvalues D2:
⎡4/3   0    0   0⎤
⎢                ⎥
⎢ 0   4/3   0   0⎥
⎢                ⎥
⎢ 0    0   4/3  0⎥
⎢                ⎥
⎣ 0    0    0   2⎦

Corresponding orthogonal matrix of eigenvectors Q:
⎡-√2   -√6   -√3       ⎤
⎢────  ────  ────   0  ⎥
⎢ 2     6     6        ⎥
⎢                      ⎥
⎢ √2   -√6   -√3       ⎥
⎢ ──   ────  ────   0  ⎥
⎢ 2     6     6        ⎥
⎢                      ⎥
⎢                  -√2 ⎥

## Configuration with coordinate $x_{45} = -4/5$ (New 2)

In [46]:
A = -sp.Rational(4, 5)
B = sp.Rational(1, 10)
C = -sp.Rational(1, 5)

X = sp.Matrix( [
    [1, A, C, C, B, B ],
    [0, 1, C, C, B, B ],
    [0, 0, 1, C, C, C ],
    [0, 0, 0, 1, C, C ],
    [0, 0, 0, 0, 1, A ],
    [0, 0, 0, 0, 0, 1 ]
] )

X = (X + X.T) - sp.eye(6) # simetriza la matriz


#######
# Matrix of coordinates on the sphere
#######
d = X.rank() # d=4

W = WfromX(X, d) # S^3 is saddle

print('\nConfiguration of points:')
sp.pprint(W)

########
# Search for negative feasible direction
#######
HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicities:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )



Eigenvalues of X
⎡0  0   0    0    0    0 ⎤
⎢                        ⎥
⎢0  0   0    0    0    0 ⎥
⎢                        ⎥
⎢0  0  6/5   0    0    0 ⎥
⎢                        ⎥
⎢0  0   0   6/5   0    0 ⎥
⎢                        ⎥
⎢0  0   0    0   9/5   0 ⎥
⎢                        ⎥
⎣0  0   0    0    0   9/5⎦

Eigenvectors of X
⎡2  -1  0   1   -1  0 ⎤
⎢                     ⎥
⎢2  -1  0   1   1   0 ⎥
⎢                     ⎥
⎢1  0   -1  -4  0   0 ⎥
⎢                     ⎥
⎢1  0   1   0   0   0 ⎥
⎢                     ⎥
⎢0  1   0   1   0   -1⎥
⎢                     ⎥
⎣0  1   0   1   0   1 ⎦

Non-null eigenvalues D2:
⎡6/5   0    0    0 ⎤
⎢                  ⎥
⎢ 0   6/5   0    0 ⎥
⎢                  ⎥
⎢ 0    0   9/5   0 ⎥
⎢                  ⎥
⎣ 0    0    0   9/5⎦

Corresponding orthogonal matrix of eigenvectors Q:
⎡       √3   -√2       ⎤
⎢ 0     ──   ────   0  ⎥
⎢       6     2        ⎥
⎢                      ⎥
⎢       √3    √2       ⎥
⎢ 0     ──    ──    0  ⎥
⎢       6     2        ⎥
⎢ 

## Configuration with coordinate $x_{45} = 1/25$ (New 3)

In [47]:
A = sp.Rational(1, 25)
B = -sp.Rational(23, 25)
C = -sp.Rational(11, 25)
D = -sp.Rational(1, 5)

X = sp.Matrix( [
    [1, A, A, D, A, B ],
    [0, 1, C, D, C, A ],
    [0, 0, 1, D, C, A ],
    [0, 0, 0, 1, D, D ],
    [0, 0, 0, 0, 1, A ],
    [0, 0, 0, 0, 0, 1 ]
] )

X = (X + X.T) - sp.eye(6) # simetriza la matriz

#######
# Matrix of coordinates on the sphere
#######
d = X.rank() # d=4

W = WfromX(X, d) # S^3 is saddle

print('\nConfiguration of points:')
sp.pprint(W)

########
# Search for negative feasible direction
#######
HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicities:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )




Eigenvalues of X
⎡0  0   0   0   0   0 ⎤
⎢                     ⎥
⎢0  0   0   0   0   0 ⎥
⎢                     ⎥
⎢0  0  6/5  0   0   0 ⎥
⎢                     ⎥
⎢           36        ⎥
⎢0  0   0   ──  0   0 ⎥
⎢           25        ⎥
⎢                     ⎥
⎢               36    ⎥
⎢0  0   0   0   ──  0 ⎥
⎢               25    ⎥
⎢                     ⎥
⎢                   48⎥
⎢0  0   0   0   0   ──⎥
⎣                   25⎦

Eigenvectors of X
⎡ 0    1   1   0   0   -1⎤
⎢                        ⎥
⎢ 1    0   1   -1  -1  0 ⎥
⎢                        ⎥
⎢ 1    0   1   1   0   0 ⎥
⎢                        ⎥
⎢3/5  2/5  -5  0   0   0 ⎥
⎢                        ⎥
⎢ 1    0   1   0   1   0 ⎥
⎢                        ⎥
⎣ 0    1   1   0   0   1 ⎦

Non-null eigenvalues D2:
⎡6/5  0   0   0 ⎤
⎢               ⎥
⎢     36        ⎥
⎢ 0   ──  0   0 ⎥
⎢     25        ⎥
⎢               ⎥
⎢         36    ⎥
⎢ 0   0   ──  0 ⎥
⎢         25    ⎥
⎢               ⎥
⎢             48⎥
⎢ 0   0   0   ──⎥
⎣             25

## Configuration with coordinate $x_{45} = 0$ (New 4) (global minima in $S^3$)

In [48]:
A = -sp.Rational(1, 2)

X = sp.Matrix( [
    [1, A, 0, 0, 0, A ],
    [0, 1, 0, 0, 0, A ],
    [0, 0, 1, A, A, 0 ],
    [0, 0, 0, 1, A, 0 ],
    [0, 0, 0, 0, 1, 0 ],
    [0, 0, 0, 0, 0, 1 ],
] )

n,n = sp.shape(X) # size of matrix
print('\nX = ')

X = (X + X.T) - sp.eye(n) # simetriza la matriz

########
# Matrix of coordinates on the sphere
########
d = X.rank() # d=4

W = WfromX(X, d) # S^3 is global minima
# W = WfromX(X, d+1) # S^4 is saddle

print('\nConfiguration of points:')
sp.pprint(W)


HL = hessL(W)
# print('\nHessian of Lagrangian:')
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicities:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )




X = 

Eigenvalues of X
⎡0  0   0    0    0    0 ⎤
⎢                        ⎥
⎢0  0   0    0    0    0 ⎥
⎢                        ⎥
⎢0  0  3/2   0    0    0 ⎥
⎢                        ⎥
⎢0  0   0   3/2   0    0 ⎥
⎢                        ⎥
⎢0  0   0    0   3/2   0 ⎥
⎢                        ⎥
⎣0  0   0    0    0   3/2⎦

Eigenvectors of X
⎡0  1  -1  0   0   -1⎤
⎢                    ⎥
⎢0  1  1   0   0   0 ⎥
⎢                    ⎥
⎢1  0  0   -1  -1  0 ⎥
⎢                    ⎥
⎢1  0  0   1   0   0 ⎥
⎢                    ⎥
⎢1  0  0   0   1   0 ⎥
⎢                    ⎥
⎣0  1  0   0   0   1 ⎦

Non-null eigenvalues D2:
⎡3/2   0    0    0 ⎤
⎢                  ⎥
⎢ 0   3/2   0    0 ⎥
⎢                  ⎥
⎢ 0    0   3/2   0 ⎥
⎢                  ⎥
⎣ 0    0    0   3/2⎦

Corresponding orthogonal matrix of eigenvectors Q:
⎡-√2               -√6 ⎤
⎢────   0     0    ────⎥
⎢ 2                 6  ⎥
⎢                      ⎥
⎢ √2               -√6 ⎥
⎢ ──    0     0    ────⎥
⎢ 2                 6  ⎥
⎢      

## Regular 5-simplex (global minima in $S^4 \subset \mathbb{R}^5$)

In [9]:
# t = -sp.Rational(1,5)

# X = sp.Matrix([
#     [1, t, t, t, t, t],
#     [t, 1, t, t, t, t],
#     [t, t, 1, t, t, t],
#     [t, t, t, 1, t, t],
#     [t, t, t, t, 1, t],
#     [t, t, t, t, t, 1]
# ])

# d = X.rank() # d=5
# W = WfromX(X, d) # matrix of points from matrix of dot products

##########
# Cartesian coordinates of points on the sphere
##########
n = 6 # number of points
d = 5 # points in R^d

W = confSimplex(n, d) # matrix with coordinates of wk in row k

print('\nConfiguration of points:')
sp.pprint(W)


# print('\nHessian of Lagrangian:')
HL = hessL(W)
# sp.pprint(HL)

########
# Eigenvalues of HL projected to tangent space
########
print('\nHL proyected onto tangent space:')
hL, Bt = projHL(HL, W)
sp.pprint(hL)

print('\nEigenvalues of projected Hessian with multiplicity:')
print( hL.eigenvals() )


#############
# matrix HL
#############
print( HL.eigenvals() )



Constructing W from X...

Matrix W of coordinates on the sphere:
⎡-√15   -√5   -√10   -√6       ⎤
⎢─────  ────  ─────  ────  -1/5⎥
⎢  5     5     10     10       ⎥
⎢                              ⎥
⎢ √15   -√5   -√10   -√6       ⎥
⎢ ───   ────  ─────  ────  -1/5⎥
⎢  5     5     10     10       ⎥
⎢                              ⎥
⎢       2⋅√5  -√10   -√6       ⎥
⎢  0    ────  ─────  ────  -1/5⎥
⎢        5     10     10       ⎥
⎢                              ⎥
⎢             3⋅√10  -√6       ⎥
⎢  0     0    ─────  ────  -1/5⎥
⎢              10     10       ⎥
⎢                              ⎥
⎢                    2⋅√6      ⎥
⎢  0     0      0    ────  -1/5⎥
⎢                     5        ⎥
⎢                              ⎥
⎣  0     0      0     0     1  ⎦

X == W * W.T?: True

Configuration of points:
⎡-√15   -√5   -√10   -√6       ⎤
⎢─────  ────  ─────  ────  -1/5⎥
⎢  5     5     10     10       ⎥
⎢                              ⎥
⎢ √15   -√5   -√10   -√6       ⎥
⎢ ───   ────  ─────  ────  -1